<a href="https://colab.research.google.com/github/sit-xinli/ai-course5/blob/main/%E3%82%B5%E3%83%9E%E3%83%A9%E3%82%A4%E3%82%B6%E3%83%BC(HuggingFace).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# １．ライブラリインストール

In [5]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub

In [8]:
from huggingface_hub import InferenceClient
from google.colab import userdata
import gradio as gr
import json
from typing import List, Dict, Tuple

## ★★★　TODO:あなたの予約する文章
default_article = """
新型コロナウイルスの感染拡大期に困窮世帯を支えるため、１兆円超の国費が投じられた特例貸付制度で、昨年末までに返済期限を迎えた２４１３億円のうち、６割にあたる１４６６億円が滞納になっていることが、厚生労働省のまとめで分かった。厚労省は今年度から、督促や困窮者に対する支援の状況を把握した上で強化を促す。生活再建を後押しし、着実な返済につなげる。
制度は、休業などで収入が減少した世帯が都道府県の社会福祉協議会（社協）を通じ、２００万円まで無利子で借りられた。１６０万世帯ほどが利用したとみられる。２０２０年３月～２２年９月の貸付額は計１兆４４３１億円に上った。
返済は２３年１月から順次始まり、１世帯あたり毎月、数千円～数万円を返済している。２４年末までに期限を迎えた総額のうち、予定通り返済されたのは３９％にあたる９４７億円。滞納額は６１％にあたる１４６６億円で２３年末（６６０億円）から倍増した。厚労省は「支援を急ぐ中、返済能力を十分に見極められなかった」と要因を説明する。
厚労省は都道府県社協に対し、市区町村社協などが行う家庭訪問による督促の状況、就労や家計改善の支援といった取り組みを毎年度、報告するよう求める。
制度では、住民税非課税などの世帯は国から返済免除を認められる。会計検査院は昨年１０月、一部の社協で免除や滞納の世帯への支援が不十分だと指摘していた。厚労省は支援状況を継続的に検証し、強化を働きかける。貸付金の返済は３４年頃まで続く見込みで、生活再建が進めば滞納状況の改善につながると期待する。
日本福祉大の角崎洋平教授（社会福祉学）は「物価高で家計は厳しさを増しており、低所得者への住宅補助など安全網も充実させるべきだ」と指摘する。
"""

## ★★★　TODO：プロンプトを""に入力する
prompt_for_summarization = """
日本語で文章を簡潔に要約して

"""

API_TOKEN=userdata.get('HF_TOKEN')
client = InferenceClient(
    model="Qwen/Qwen2.5-1.5B-Instruct",
    token=API_TOKEN
)

response = client.chat_completion(
            messages=[
                 {"role": "system", "content": prompt_for_summarization},
                 {"role": "user", "content": default_article}
            ],
            temperature = 1.0,
            max_tokens=200,
    )
print("Summary:", response.choices[0].message["content"])


Summary: 日本の感染症対策として、政府による一連の措置により多くの困窮世帯が支援を受けることが伝えられています。しかし、これらのプログラムにおいて、一部の世帯では返済が遅れていることが明らかになりました。今後、支援を行っている自治体からの督促や緊急的な生活改善支援を増やすことで、滞納を解消し返済を確保することは重要です。また、住民税等非課税世帯を含む特殊世帯にも支援を提供することにより、困窮世帯の負債解消と回復を助けます。

この制度によって2020年3月から2022年9月までの総借金額は1兆4千4百三十億円となり、滞納世帯で占める241億円のうち、6割程度という結果になっています。そして、39%が返


# ２．要約：次のセルは実行して出力ファイルがえられたら、手動で停止してください

In [14]:
# 会話をリセットする関数
def reset() -> List[Tuple[str, str]]:
    return []

# 関数を呼び出してモデルを生成する。
def interact_summarization(prompt: str, article: str, temp = 1.0) -> List[Tuple[str, str]]:
    '''
    * 引数
      - prompt: このセクションで使用するプロンプト。
      - article: 要約する記事
      - temp: このモデルの温度パラメータ。 温度はチャットボットの出力をコントロールするために使われる。
              温度が高ければ高いほど、よりクリエイティブな反応が得られる。
    '''
    response = client.chat_completion(
            messages=[
                 {"role": "system", "content": prompt},
                 {"role": "user", "content": article}
            ],
            temperature = temp,
            max_tokens=300,
    )

    result: List[Dict[str, str]] = []
    result.append({"role": "user", "content": article})
    result.append({"role": "assistant", "content": response.choices[0].message["content"]})
    return result

# 会話全体を出力する関数
def export_summarization(chatbot: List[Tuple[str, str]], article: str) -> None:
    '''
    * 引数
      - chatbot: モデル自身。会話はタプルのリストに格納される。
      - article: 要約される記事
    '''
    target = {"chatbot": chatbot, "article": article}
    with open("summarized.json", "w", encoding='utf-8') as file:
        json.dump(target, file, ensure_ascii=False)

# この部分はGradioのユーザインターフェイスを生成する
with gr.Blocks() as demo:
    gr.Markdown("# 好きな記事を記入して、チャットボットが要約してくれるよ！")
    chatbot = gr.Chatbot(type='messages')
    prompt_textbox = gr.Textbox(label="プロンプト", value=prompt_for_summarization, visible=False)
    article_textbox = gr.Textbox(label="文章", interactive = True, value = default_article)
    with gr.Column():
        gr.Markdown("#  Temperature(温度)はチャットボットの出力を制御するために使用される。 温度が高ければ高いほど、よりクリエイティブな反応が得られる。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step = 0.1, label="温度")
    with gr.Row():
        sent_button = gr.Button(value="送信")
        reset_button = gr.Button(value="リセット")

    with gr.Column():
        gr.Markdown("# 満足のいく結果が得られたら。 「出力」ボタンをクリックしてファイル化してください。")
        export_button = gr.Button(value="出力")
    sent_button.click(interact_summarization, inputs=[prompt_textbox, article_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_summarization, inputs=[chatbot, article_textbox])

demo.launch(debug = True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3618c265b5ae9ff728.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3618c265b5ae9ff728.gradio.live


# ３．出力：前のセルの実行を停止してから、次のセルを実行する。結果を採点システムにコピーする。

In [15]:
# load the conversation log json file
with open("summarized.json", "r") as f:
    context = json.load(f)

summarization = context['chatbot'][-1]['content']
article = context['article']

# this part constructs the Gradio UI interface
with gr.Blocks() as demo:
    gr.Markdown("# 好きな記事を要約してくれたよ！")
    with gr.Column():
        gr.Markdown("# この部分を採点システムにコピーする.")
        gr.Textbox(label = "文章", value = article, show_copy_button = True)
        gr.Textbox(label="要約", value = summarization, show_copy_button = True)

demo.launch(debug = True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a7b05f349c6b3439b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a7b05f349c6b3439b8.gradio.live


# ４. セルの実行を停止したことを確認してください